In [ ]:
import brightway2 as bw
import numpy as np
from pathlib import Path
import pickle, json
from pypardiso import spsolve

#Local files
from import_databases import *
from utils_exiobase import exiobaseLCA

write_dir = Path("write_files")

# 1. Get exiobase scores

In [ ]:
bw.projects.set_current("exiobase_industry_workaround")

In [ ]:
exiobase_name = "EXIOBASE 2.2"
ex = bw.Database(exiobase_name)
methods = [
#     ('ReCiPe Endpoint (H,A)', 'resources', 'total'),
#     ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total'),
#     ('ReCiPe Endpoint (H,A)', 'human health', 'total'),
#     ('USEtox', 'ecotoxicity', 'total'),
#     ('USEtox', 'human toxicity', 'total'),
]
methods_save_dict = {
    ('ReCiPe Endpoint (H,A)', 'resources', 'total'): 
        ('ReCiPe Endpoint (H,A) (obsolete)', 'resources', 'total'),
    ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total'): 
        ('ReCiPe Endpoint (H,A) (obsolete)', 'ecosystem quality', 'total'),
    ('ReCiPe Endpoint (H,A)', 'human health', 'total'): 
        ('ReCiPe Endpoint (H,A) (obsolete)', 'human health', 'total'),
    ('USEtox', 'ecotoxicity', 'total'): 
        ('USEtox (obsolete)', 'ecotoxicity', 'total'),
    ('USEtox', 'human toxicity', 'total'): 
        ('USEtox (obsolete)', 'human toxicity', 'total'),
}

In [ ]:
%%time
# takes 20 min or so for one lcia method, 22 min for 5 methods
filepath = write_dir / "exiobase_lca.pickle"
if not filepath.exists():
    exiobase_scores_industry_workaround = {}
    for i,act in enumerate(ex):
        if i%500==0:
            print("lca for act {0:4d}/{1}".format(i,len(ex)))
        db_code = (act.as_dict()["database"], act.as_dict()["code"])
        demand = {act: 1}
        lca = bw.LCA(demand, methods[0])
        lca.lci()
        lca.lcia()
        exiobase_scores_industry_workaround[db_code] = {}
        for method in methods:
            lca.switch_method(method)
            lca.redo_lcia()
            exiobase_scores_industry_workaround[db_code].update({
                methods_save_dict[method]: lca.score
            })
    with open(filepath, 'wb') as f:
        pickle.dump(exiobase_scores_industry_workaround, f)
else:
    with open(filepath, 'rb') as f:
        exiobase_scores_industry_workaround = pickle.load(f)

In [ ]:
exiobase_scores_industry_workaround

# 2. Import databases

In [ ]:
which_pc = "sasha"

project = "rebound"
bw.projects.set_current(project)

In [ ]:
# del bw.databases['Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere']
# del bw.databases['Agribalyse 1.3 - ecoinvent 3.6 cutoff']
# del bw.databases['CH consumption 1.0']

In [ ]:
if which_pc == 'sasha':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    # Newer databases
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # ex34_path = '/Users/akim/Documents/LCA_files/exiobase 3.4/IOT_2011_pxp/' #TODO
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'

In [ ]:
%%time 
# This takes a while
ei36_name = 'ecoinvent 3.6 cutoff'
co_name = CONSUMPTION_DB_NAME

if project == 'rebound':
    bw.bw2setup()
    import_exiobase_22(ex22_path)
    import_ecoinvent(ei36_path, ei36_name)
    import_agribalyse_13(ag13_path, ei36_name)
    # Consumption DB
    create_ecoinvent_33_project(ei33_path)
    exclude_dbs = [
        'heia', 
    ]
    import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
    add_consumption_activities(co_name, habe_path)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)

# 3. Compute total score for a given demand

In [ ]:
# file = "write_files/exiobase_lca_gwp.pickle"
# with open(file, 'rb') as f:
#     exiobase_scores_industry_workaround = pickle.load(f) 

In [ ]:
co = bw.Database(CONSUMPTION_DB_NAME)
demand_act = co.search('ch hh average consumption')
assert len(demand_act) == 1
demand = {demand_act[0]: 1}
sectors = [act for act in co if 'sector' in act["name"]]

In [ ]:
%%time
# This takes 30s or so
exio_lca = exiobaseLCA(
    project, 
    demand,
    exiobase_scores_industry_workaround,
)
scores = exio_lca.compute_total_scores()
scores

# Contribution analysis

In [ ]:
%%time
project = 'rebound'
bw.projects.set_current(project)
co = bw.Database("CH consumption 1.0")
contribution_scores = {}
i = 0
# for act in sectors:
for act in list(co):
    demand = {act: 1}
    exio_lca = exiobaseLCA(
        project, 
        demand,
        exiobase_scores_industry_workaround,
    )
    act_name = act["name"]
    score = exio_lca.compute_total_scores()
    contribution_scores[act_name] = score
#     print(i, score[methods[0]], act)
    print(i)
    i+=1

In [ ]:
# contribution_scores2 = {k.as_dict()['name']: v[methods[0]] for k,v in contribution_scores.items()}
# contribution_scores_sorted = dict(sorted(contribution_scores2.items(), key=lambda x: x[1], reverse=True))
# contribution_scores_sorted

In [ ]:
# filepath = write_dir / "contribution_scores_str.pickle"
# with open(filepath, 'wb') as f:
#     pickle.dump(contribution_scores, f)

In [ ]:
contribution_scores

In [ ]:
filepath = write_dir / "contribution_scores_usetox.pickle"
with open(filepath, 'wb') as f:
    pickle.dump(contribution_scores, f)

# Contribution analysis for sectors

In [ ]:
project = 'rebound'
bw.projects.set_current(project)
co = bw.Database("CH consumption 1.0")
sectors = [act for act in co if 'sector' in act["name"]]
contribution_scores = {}
for act in sectors:
    demand = {act: 1}
    exio_lca = exiobaseLCA(
        project, 
        demand,
        exiobase_scores_industry_workaround,
    )
    act_name = act["name"]
    score = exio_lca.compute_total_scores()
    contribution_scores[act_name] = score
    print(score, act)


In [ ]:
# contribution_scores2 = {k.as_dict()['name'].replace('sector', ''): v for k,v in contribution_scores.items()}
# contribution_scores2

In [ ]:
# pd.DataFrame.from_dict(contribution_scores2).T

In [ ]:
# contribution_scores_sorted = dict(sorted(contribution_scores2.items(), key=lambda x: x[1], reverse=True))
# contribution_scores_sorted

In [ ]:
# filepath = write_dir / "contribution_scores_sectors.pickle"
# with open(filepath, 'wb') as f:
#     pickle.dump(contribution_scores_sorted, f)

In [ ]:
filepath = write_dir / "contribution_scores_sectors_usetox.pickle"
with open(filepath, 'wb') as f:
    pickle.dump(contribution_scores, f)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure([
    go.Bar(
        x=list(contribution_scores_sorted.values()), 
        y=y, 
        orientation='h',
    )
])
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(title='LCIA scores, [kg CO2-eq]')
fig.update_layout(
    width=650,
    height=500,
#     margin=dict(l=500,r=20,t=0,b=0,)
)
fig.show()
fig.write_image('contribution_analysis_sectors.pdf')

# Contributions of specific categories

In [ ]:
co = bw.Database(CONSUMPTION_DB_NAME)
demand_act = co.search('ch hh average consumption')
assert len(demand_act) == 1
demand_act = demand_act[0]

purchase, transportation_services, transport_by_air, recreation, package_holidays = {}, {}, {}, {}, {}
purchase_name = 'Purchase and operation of vehicles'
transportation_services_name = "Transportation services"
transport_by_air_name = "Passenger transport by air"
recreation_name = "Recreation and culture"
package_holidays_name = "Package holidays"
for exc in demand_act.exchanges():
    if exc.input.get('category_middle') == purchase_name:
        purchase.update({
            exc.input: exc.amount,
        })
    if exc.input.get('category_middle') == transportation_services_name:
        if exc.input.get('category_fine') == transport_by_air_name:
            transport_by_air.update({
                exc.input: exc.amount,
            })
        else:
            transportation_services.update({
                exc.input: exc.amount,
            })
    if exc.input.get('category_coarse') == recreation_name:
        if exc.input.get('category_middle') == package_holidays_name:
            package_holidays.update({
                exc.input: exc.amount,
            })
        else:
            recreation.update({
                exc.input: exc.amount,
            })
fus = {
    purchase_name: purchase,
    transportation_services_name: transportation_services,
    transport_by_air_name: transport_by_air,
    recreation_name: recreation,
    package_holidays_name: package_holidays,
} 

In [ ]:
contribution_scores = {}
for name,demand in fus.items():
    exio_lca = exiobaseLCA(
        project, 
        demand,
        exiobase_scores_industry_workaround,
    )
    score = exio_lca.compute_total_scores()
    exchanges = {}
    for exc, exc_amount in demand.items():
        exchanges[exc['name']] = "{} {}".format(exc_amount, exc.get('unit'))
    contribution_scores[name] = {
        "LCA score": score,
        "Exchanges": exchanges,
    }
    print(score, name)

In [ ]:
path_scores = 'write_files/contribution_scores_5categories_usetox.pickle'
with open(path_scores, 'wb') as f:
    pickle.dump(contribution_scores, f)

# Contribution scores, names of activities, not bw

In [ ]:
import brightway2 as bw
import pickle

In [ ]:
which_pc = "sasha"

project = "rebound"
bw.projects.set_current(project)
methods = [
    ('IPCC 2013', 'climate change', 'GWP 100a'),
]

path_scores = 'write_files/contribution_scores.pickle'
with open(path_scores, 'rb') as f:
    scores = pickle.load(f)

In [ ]:
scores2 = {k.as_dict()['name']: v[methods[0]] for k,v in scores.items()}
scores_sorted = dict(sorted(scores2.items(), key=lambda x: x[1], reverse=True))

In [ ]:
path_scores_str = 'write_files/contribution_scores_str.pickle'
with open(path_scores_str, 'wb') as f:
    pickle.dump(scores_sorted, f)